In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from multiprocessing import Pool
import time
from difflib import SequenceMatcher
from haversine import haversine, Unit
from math import radians

import numpy as np

def GET_DATAFRAME():
    host_url = "db.ds.mycelebs.com"
    user_nm = "celebDev"
    passwd = "vmfhwprxmwlzhqk2@#"
    port_num = 3306
    db_name = "staypia"
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    with engine.connect() as connection:
        gg_df = pd.read_sql('select hotelid, Name, Address, Latidude, Longitude, city from gg_hotels', con=connection)
        hc_df = pd.read_sql(
            'select HotelID, CountryFileName, CountryName, PlaceFileName, HotelName, HotelAddress, Latitude, Longitude from hotels',
            con=connection)
        
    # --------------------------------------------------------------------    
    # 입력 파일 이름 사용자 맘대로 변경
    gg_add_df = pd.read_csv('gg_hotels_add_result.csv', index_col=0)
    # --------------------------------------------------------------------
    
    
    
    gg_add_df = gg_add_df[gg_add_df['hc_name_1'].notnull()]
    gg_add_df.reset_index(inplace=True)
    gg_add_df['hc_name_1'] = gg_add_df['hc_name_1'].str.upper()
    gg_except_df = gg_add_df[gg_add_df['hc_name_1'].isnull()]
    hc_df['PlaceFileName'] = hc_df['PlaceFileName'].str.replace('_', ' ')
    gg_df['city'] = gg_df['city'].str.replace(' - ALL LOCATIONS', '')
    hc_df['PlaceFileName'] = hc_df['PlaceFileName'].str.upper()
    hc_df['HotelName'] = hc_df['HotelName'].str.upper()

    return gg_df, hc_df, gg_add_df, gg_except_df
def match_hotels(text):
    global i
    global gg_list
    global match_dic
    global final_df
    try:
        if text in hc_list:
            gg_idx = int(gg_add_df['gg_hotelid'][gg_list.index(text)])
            hc_idx = hc_df['HotelID'][hc_list.index(text)]
            gg_idx_df = gg_df[gg_df['hotelid'] == gg_idx]
            hc_idx_df = hc_df[hc_df['HotelID'] == hc_idx]
            match_dic = {'hm_hi_code' : '', 'hm_sup_hotelcode': '', 'hmsup_code': '', 'Country_eng': '', 'Country_ko': '', 'hc_city': '', 'hc_hotelname': '', 'hc_address': '', 'hc_lat': '', 'hc_lng': '', 'hotelname': '', 'address': '','lat': '', 'lng': ''}
            match_dic['hm_hi_code'] = hc_idx
            match_dic['hm_sup_hotelcode'] = gg_idx
            match_dic['hmsup_code'] = 'SUPLIST_GO'
            match_dic['Country_eng'] = hc_idx_df['CountryFileName'].values[0]
            match_dic['Country_ko'] = hc_idx_df['CountryName'].values[0]
            match_dic['hc_city'] = hc_idx_df['PlaceFileName'].values[0]
            match_dic['hc_hotelname'] = hc_idx_df['HotelName'].values[0]
            match_dic['hc_address'] = hc_idx_df['HotelAddress'].values[0]
            match_dic['hc_lat'] = hc_idx_df['Latitude'].values[0]
            match_dic['hc_lng'] = hc_idx_df['Longitude'].values[0]
            match_dic['hotelname'] = gg_idx_df['Name'].values[0]
            match_dic['address'] = gg_idx_df['Address'].values[0]
            match_dic['lat'] = gg_idx_df['Latidude'].values[0]
            match_dic['lng'] = gg_idx_df['Longitude'].values[0]
        else:
            gg_idx = int(gg_add_df['gg_hotelid'][gg_list.index(text)])
            gg_idx_df = gg_df[gg_df['hotelid'] == gg_idx]
            match_dic = {'hm_hi_code' : '', 'hm_sup_hotelcode': '', 'hmsup_code': '', 'Country_eng': '', 'Country_ko': '', 'hc_city': '', 'hc_hotelname': '', 'hc_address': '', 'hc_lat': '', 'hc_lng': '', 'hotelname': '', 'address': '','lat': '', 'lng': ''}
            match_dic['hm_sup_hotelcode'] = gg_idx
            match_dic['hmsup_code'] = 'SUPLIST_GO'
            match_dic['hotelname'] = gg_idx_df['Name'].values[0]
            match_dic['address'] = gg_idx_df['Address'].values[0]
            match_dic['lat'] = gg_idx_df['Latidude'].values[0]
            match_dic['lng'] = gg_idx_df['Longitude'].values[0]
            print(text, 'no search')
    except:
        match_dic = {'hm_hi_code': '', 'hm_sup_hotelcode': '', 'hmsup_code': '', 'Country_eng': '', 'Country_ko': '',
                     'hc_city': '', 'hc_hotelname': '', 'hc_address': '', 'hc_lat': '', 'hc_lng': '', 'hotelname': '',
                     'address': '', 'lat': '', 'lng': ''}
        print(text)
    i +=1
    return match_dic


gg_df, hc_df, gg_add_df, gg_except_df = GET_DATAFRAME()
gg_list = gg_add_df['hc_name_1'].to_list()
hc_list = hc_df['HotelName'].to_list()

start_time = time.time()
i = 1
pool = Pool(processes=4)
final_list = pool.map(match_hotels, gg_list)
print('걸린 시간 : %s' % (time.time() - start_time))
final_df = pd.DataFrame(final_list, columns=['hm_hi_code', 'hm_sup_hotelcode', 'hmsup_code', 'Country_eng', 'Country_ko', 'hc_city', 'hc_hotelname', 'hc_address', 'hc_lat', 'hc_lng', 'hotelname', 'address','lat', 'lng'])

match_list = list(map(lambda x : SequenceMatcher(None, final_df['hc_hotelname'][x], final_df['hotelname'][x]).ratio(), range(0,final_df.shape[0])))


df2_dic = {'match_rate' : match_list}
df2 = pd.DataFrame(df2_dic, columns=['match_rate'])
final_df = pd.merge(final_df, df2, left_index=True, right_index=True, how='left')


# --------------------------------------------------------------------
# 출력 파일 이름 사용자 맘대로 변경
final_df.to_excel('match_result.xlsx', encoding='utf-8-sig')
# --------------------------------------------------------------------
